In [9]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras import layers, Model
from tensorflow.keras.layers import Dense, Conv2D, Conv2DTranspose, Flatten, Reshape, LeakyReLU, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

In [6]:
#Data Preprocessing
def preprocess(image, label):
    image = tf.image.resize(image, [64, 64])  # Resize images to 64x64
    image = tf.cast(image, tf.float32) / 255.0  # Normalize images to [0, 1]
    return image

#Loading the Dataset
(ds_train, ds_val, ds_test), ds_info = tfds.load(
    'malaria',
    split=['train[:70%]', 'train[70%:85%]', 'train[85%:]'],
    as_supervised=True,
    with_info=True,
)

batch_size = 32
ds_train = ds_train.map(preprocess).shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
ds_val = ds_val.map(preprocess).batch(batch_size).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.map(preprocess).batch(batch_size).prefetch(tf.data.AUTOTUNE)


In [7]:
#Defining the Generator and Discriminator
def make_generator_model():
    model = tf.keras.Sequential([
        Dense(8*8*256, use_bias=False, input_shape=(100,)),
        BatchNormalization(),
        LeakyReLU(),
        tf.keras.layers.Reshape((8, 8, 256)),
        Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False),
        BatchNormalization(),
        LeakyReLU(),
        Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        BatchNormalization(),
        LeakyReLU(),
        Conv2DTranspose(32, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        BatchNormalization(),
        LeakyReLU(),
        Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')
    ])
    return model

def make_discriminator_model():
    model = tf.keras.Sequential([
        Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[64, 64, 3]),
        LeakyReLU(),
        Dropout(0.3),
        Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
        LeakyReLU(),
        Dropout(0.3),
        Flatten(),
        Dense(1)
    ])
    return model

In [14]:
#Defining Loss and Optimizers
cross_entropy = BinaryCrossentropy(from_logits=True)

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

generator = make_generator_model()
discriminator = make_discriminator_model()

generator_optimizer = Adam(1e-4)
discriminator_optimizer = Adam(1e-4)


In [15]:
#Defining the Training and Validation Steps
@tf.function
def train_step(images):
    noise = tf.random.normal([batch_size, 100])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

@tf.function
def val_step(images):
    noise = tf.random.normal([batch_size, 100])
    generated_images = generator(noise, training=False)
    fake_output = discriminator(generated_images, training=False)
    val_loss = discriminator_loss(tf.ones_like(fake_output), fake_output)
    return val_loss

In [16]:
# Step 6: Execute the Training Loop
def train(dataset, epochs):
    for epoch in range(epochs):
        for image_batch in dataset:
            train_step(image_batch)

        val_losses = []
        for val_image_batch in ds_val:
            loss = val_step(val_image_batch)
            val_losses.append(loss)

        print(f"Epoch {epoch + 1}, Validation Loss: {tf.reduce_mean(val_losses).numpy()}")

# Step 7: Visualize Training Progress
def plot_losses(val_losses):
    plt.figure(figsize=(10, 5))
    plt.plot(val_losses, label='Validation Loss')
    plt.title('Validation Loss Over Time')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

# Initialize and train the GAN with validation
epochs = 10
train(ds_train, epochs)

Epoch 1, Validation Loss: 0.4832709729671478
Epoch 2, Validation Loss: 0.9257936477661133
Epoch 3, Validation Loss: 0.5708110332489014
Epoch 4, Validation Loss: 0.38087767362594604
Epoch 5, Validation Loss: 0.528873860836029
Epoch 6, Validation Loss: 0.5845460891723633
Epoch 7, Validation Loss: 0.48252153396606445
Epoch 8, Validation Loss: 0.3405482769012451
Epoch 9, Validation Loss: 0.6675868034362793
Epoch 10, Validation Loss: 0.4260174036026001
